In [1]:
pip install google-api-python-client

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install youtube-search-python

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
pip install tqdm

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
pip install httpx==0.27.2

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.0.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from youtubesearchpython import VideosSearch
import pandas as pd
import time
from datetime import datetime
from tqdm import tqdm

In [10]:
def search_youtube_video_id(title, artist, max_retries=3):
    """
    Cerca un video YouTube e ritorna il video ID
    """
    query = f"{artist} {title} official video"
    
    for attempt in range(max_retries):
        try:
            videos_search = VideosSearch(query, limit=1)
            results = videos_search.result()
            
            if results and 'result' in results and len(results['result']) > 0:
                video_id = results['result'][0]['id']
                return video_id
            else:
                return None
                
        except Exception as e:
            if attempt < max_retries - 1:
                time.sleep(1)  # Pausa prima di riprovare
                continue
            else:
                print(f"Errore per '{artist} - {title}': {e}")
                return None
    
    return None

def add_youtube_ids_to_dataframe(df, title_col='title', artist_col='primary_artist', 
                                  checkpoint_every=100, checkpoint_file='youtube_ids_checkpoint.csv'):
    """
    Aggiunge colonna youtube_video_id al dataframe con progress bar tqdm
    """
    # Crea la colonna se non esiste
    if 'youtube_video_id' not in df.columns:
        df['youtube_video_id'] = None
    
    total = len(df)
    start_time = datetime.now()
    found_count = 0
    
    print(f"Inizio scraping di {total} canzoni...")
    print(f"Ora di inizio: {start_time.strftime('%H:%M:%S')}\n")
    
    # Usa tqdm per iterare sul dataframe
    for idx, row in tqdm(df.iterrows(), total=total, 
                         desc="Scraping YouTube IDs", 
                         unit="song",
                         colour="green",
                         ncols=100):
        
        # Salta se già processato
        if pd.notna(df.at[idx, 'youtube_video_id']):
            found_count += 1
            continue
        
        # Cerca il video ID
        video_id = search_youtube_video_id(row[title_col], row[artist_col])
        df.at[idx, 'youtube_video_id'] = video_id
        
        if video_id:
            found_count += 1
        
        # Checkpoint periodico
        if idx % checkpoint_every == 0 and idx > 0:
            df.to_csv(checkpoint_file, index=False)
            tqdm.write(f"✓ Checkpoint salvato: {found_count}/{idx+1} trovati")
        
        # Rate limiting
        time.sleep(1.5)
    
    # Salvataggio finale
    df.to_csv(checkpoint_file, index=False)
    
    elapsed_total = (datetime.now() - start_time).total_seconds() / 60
    
    print(f"\n{'='*60}")
    print(f"✓ Scraping completato!")
    print(f"Tempo totale: {elapsed_total:.1f} minuti")
    print(f"Video ID trovati: {found_count}/{total} ({found_count/total*100:.1f}%)")
    print(f"{'='*60}\n")
    
    return df

In [11]:
df = pd.read_csv('songs_with_youtube_ids.csv')

In [12]:
df_ids = add_youtube_ids_to_dataframe(
        df, 
        title_col='title', 
        artist_col='primary_artist',
        checkpoint_every=100,
        checkpoint_file='songs_with_youtube_ids.csv'
        )

Inizio scraping di 11166 canzoni...
Ora di inizio: 16:23:47



Scraping YouTube IDs:  16%|█████▏                          | 1800/11166 [04:57<7:30:31,  2.89s/song]

✓ Checkpoint salvato: 1798/1801 trovati


Scraping YouTube IDs:  17%|█████▍                          | 1900/11166 [09:55<7:35:16,  2.95s/song]

✓ Checkpoint salvato: 1898/1901 trovati


...

✓ Checkpoint salvato: 10977/11001 trovati


Scraping YouTube IDs:  99%|██████████████████████████████▊| 11100/11166 [7:05:19<02:46,  2.53s/song]

✓ Checkpoint salvato: 11077/11101 trovati


Scraping YouTube IDs: 100%|███████████████████████████████| 11166/11166 [7:08:07<00:00,  2.30s/song]


✓ Scraping completato!
Tempo totale: 428.1 minuti
Video ID trovati: 11142/11166 (99.8%)

